<div style='float: right'><img src='pic/fukumen.png'/></div>
## <div id='fukumen' />覆面算

In [ ]:
import numpy as np
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
 SEND
 MORE
-----
MONEY""".split('\n')
data = data[:-2] + [data[-1]]
nw, nh = len(data[0]), len(data)
r10 = range(10)

### 問題
* 各記号は0から9までの数字の1に対応します
  * 同じ記号には同じ数字が対応します
  * 最上位の桁は0になりません

### 変数
* v:各位置でどの数字を使うか (1)
* r:各位置の1桁の数字 (2)
* q:nw桁の数字 (3)

### 制約
* $v_{ij}$内で数字は１つのみ (4)
* rをvで表現 (5)
* 空白は0、先頭は0でない (6)
* 同じ文字は同じ数字、違う文字は違う数字 (7)
* qをrで表現 (8)
* qの最後以外の合計は、qの最後と等しい (9)

In [ ]:
m = LpProblem()
v = addbinvars(nh, nw, 10) # (1)
r = addvars(nh, nw) # (2)
q = addvars(nh) # (3)
dic = {}
for i in range(nh):
    e = LpAffineExpression()
    fst = True
    for j in range(nw):
        c = data[i][j]
        e = e * 10 + r[i][j]
        m += lpSum(v[i][j]) == 1 # (4)
        m += lpDot(r10, v[i][j]) == r[i][j] # (5)
        if c == ' ':
            m += v[i][j][0] == 1 # (6)
        else:
            if c in dic:
                m += lpDot(r10, v[i][j]) == lpDot(r10, dic[c]) # (7)
            else:
                dic[c] = v[i][j]
            if fst:
                fst = False
                m += v[i][j][0] == 0 # (6)
    m += e == q[i] # (8)
for i, t in dic.items():
    for j, u in dic.items():
        if i < j:
            for k in r10:
                m += t[k] + u[k] <= 1 # (7)
m += lpSum(q[:-1]) == q[-1] # (9)
%time m.solve()
np.vectorize(value)(r).astype(int)